# Trial Code

In [1]:
from itertools import groupby
from pycocotools import mask as mutils
import numpy as np
from tqdm import tqdm
import pandas as pd
import os
import pickle
import cv2
from multiprocessing import Pool
import matplotlib.pyplot as plt

In [3]:
exp_name = "v3"
conf_name = "mask_rcnn_s101_fpn_syncbn-backbone+head_mstrain_1x_coco"
cell_mask_dir = '../input/hpa-mask/hpa_cell_mask'    
ROOT = '../input/hpa-single-cell-image-classification/'
train_or_test = 'train'
img_dir = f'../work/mmdet_{exp_name}_{train_or_test}'
!mkdir -p {img_dir}
df = pd.read_csv(os.path.join(ROOT, 'train.csv'))

# this script takes more than 9hours for full data.
debug = True
if debug:
    df = df[:4]

In [4]:
print(df)

                                     ID  ImageWidth  ImageHeight  \
0  0040581b-f1f2-4fbe-b043-b6bfea5404bb        2048         2048   
1  004a270d-34a2-4d60-bbe4-365fca868193        2048         2048   
2  00537262-883c-4b37-a3a1-a4931b6faea5        2048         2048   

           PredictionString  
0  0 1 eNoLCAgIMAEABJkBdQ==  
1  0 1 eNoLCAgIMAEABJkBdQ==  
2  0 1 eNoLCAgIMAEABJkBdQ==  


## Helper Functions

In [5]:
#The '->' at the end of the function is function annotation. 
#The typing package interprets the meaning of the function

#The same goes for the input. It is just saying it takes a 'mask' that is an np.ndarray.
#t.Text is the output of the function. 

#COCO is a large-scale object detection, segmentation, and captioning dataset. Its API
# has masking tools. https://cocodataset.org/#home

def encode_binary_mask(mask: np.ndarray) -> t.Text:
  """Converts a binary mask into OID (OpenImage?) challenge encoding ascii text."""

  # check input mask --
  if mask.dtype != np.bool:
    raise ValueError(
        "encode_binary_mask expects a binary mask, received dtype == %s" %
        mask.dtype)

  mask = np.squeeze(mask)
  if len(mask.shape) != 2:
    raise ValueError(
        "encode_binary_mask expects a 2d mask, received shape == %s" %
        mask.shape)

  # convert input mask to expected COCO API input --
  mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
  mask_to_encode = mask_to_encode.astype(np.uint8)
  mask_to_encode = np.asfortranarray(mask_to_encode)

  # RLE encode mask --
  encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

  # compress and base64 encoding -- Encodes binary data to printable ASCII characters
  binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
  base64_str = base64.b64encode(binary_str)
  return base64_str.decode()

In [6]:
#Assert raises AssertionError is value is False. Used for debugging

#cv2.IMREAD_UNCHANGED returns image as-is with alpha channel

#Change image to 'uint8' to keep values in 0-255 range.

def read_img(image_id, color, train_or_test='train', image_size=None):
    filename = f'{ROOT}/{train_or_test}/{image_id}_{color}.png'
    assert os.path.exists(filename), f'not found {filename}'
    img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    if image_size is not None:
        img = cv2.resize(img, (image_size, image_size))
    if img.dtype == 'uint16':
        img = (img/256).astype('uint8') 
    return img

In [7]:
#I believe the (1,2,0) for the transpose function is the order. 
# It might mean change to green, blue, red but not sure why yet. 

def load_RGBY_image(image_id, train_or_test='train', image_size=None):
    red = read_img(image_id, "red", train_or_test, image_size)
    green = read_img(image_id, "green", train_or_test, image_size)
    blue = read_img(image_id, "blue", train_or_test, image_size)
    # using rgb only here
    #yellow = read_img(image_id, "yellow", train_or_test, image_size)
    stacked_images = np.transpose(np.array([red, green, blue]), (1,2,0))
    return stacked_images

In [8]:
def print_masked_img(image_id, mask):
    img = load_RGBY_image(image_id, train_or_test)
    
    plt.figure(figsize=(15, 15))
    plt.subplot(1, 3, 1)
    plt.imshow(img)
    plt.title('Image')
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(mask)
    plt.title('Mask')
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.imshow(img)
    plt.imshow(mask, alpha=0.6)
    plt.title('Image + Mask')
    plt.axis('off')
    plt.show()

## Generate files for mmdetection

MMDetection is an open source object detection toolbox based on PyTorch. It is a part of the OpenMMLab project developed by Multimedia Laboratory, CUHK.

In [9]:
out_image_dir = f'/home/chad/GitHub/MMDetection/mmdetection/work/mmdet_{exp_name}_{train_or_test}/'
!mkdir -p {out_image_dir}

annos = []
for idx in tqdm(range(len(df))):
    image_id = df.iloc[idx].ID
    img = load_RGBY_image(image_id, train_or_test)
    
    cv2.imwrite(f'{out_image_dir}/{image_id}.jpg', img)
    ann = {
        'filename': image_id+'.jpg',
        'width': img.shape[1],
        'height': img.shape[0],
        'ann': {
            'bboxes': None,
            'labels': None,
            'masks': None
        }
    }
    annos.append(ann)
    
with open(f'/home/chad/GitHub/MMDetection/mmdetection/work/mmdet_{exp_name}_tst.pkl', 'wb') as f:
    pickle.dump(annos, f)

100%|██████████| 3/3 [00:00<00:00,  3.98it/s]


## Inference

In [14]:
import os 
os.getcwd()

'/home/chad/GitHub/MMDetection/mmdetection'

In [34]:
#exp_name = "v4"
#conf_name = "mask_rcnn_s101_fpn_syncbn-backbone+head_mstrain_1x_coco"
#model_name = 'mask_rcnn_resnest101_v5_ep9'

config = f'configs/hpa_{exp_name}/{conf_name}.py'
print(config)

configs/hpa_v4/mask_rcnn_s101_fpn_syncbn-backbone+head_mstrain_1x_coco.py


In [35]:
model_file = f'../input/hpa-models/{model_name}.pth'
print(model_file)

../input/hpa-models/mask_rcnn_resnest101_v5_ep9.pth


In [31]:
result_pkl = f'../work/{model_name}.pkl'


In [32]:
additional_conf = '--cfg-options'
additional_conf += ' test_cfg.rcnn.score_thr=0.001'


In [33]:
cmd = f'python tools/test.py {config} {model_file} --out {result_pkl} {additional_conf}'
!cd ../mmdetection; {cmd}


Traceback (most recent call last):
  File "/home/chad/anaconda3/lib/python3.8/site-packages/mmcv/utils/registry.py", line 179, in build_from_cfg
    return obj_cls(**args)
  File "/home/chad/anaconda3/lib/python3.8/site-packages/mmdet/datasets/custom.py", line 87, in __init__
    self.data_infos = self.load_annotations(self.ann_file)
  File "/home/chad/anaconda3/lib/python3.8/site-packages/mmdet/datasets/custom.py", line 112, in load_annotations
    return mmcv.load(ann_file)
  File "/home/chad/anaconda3/lib/python3.8/site-packages/mmcv/fileio/io.py", line 41, in load
    obj = handler.load_from_path(file, **kwargs)
  File "/home/chad/anaconda3/lib/python3.8/site-packages/mmcv/fileio/handlers/pickle_handler.py", line 13, in load_from_path
    return super(PickleHandler, self).load_from_path(
  File "/home/chad/anaconda3/lib/python3.8/site-packages/mmcv/fileio/handlers/base.py", line 20, in load_from_path
    with open(filepath, mode) as f:
FileNotFoundError: [Errno 2] No such file or d

In [ ]:
result = pickle.load(open('../mmdetection/'+result_pkl, 'rb'))

## Result Check

In [23]:
for ii in range(3):
    image_id = annos[ii]['filename'].replace('.jpg','').replace('.png','')
    for class_id in range(19):
        #print(ii,class_id,len(result[ii][0][class_id]), len(result[ii][1][class_id]))
        bbs = result[ii][0][class_id]
        sgs = result[ii][1][class_id]
        for bb, sg in zip(bbs,sgs):
            box = bb[:4]
            cnf = bb[4]
            h = sg['size'][0]
            w = sg['size'][0]
            if cnf > 0.3:
                print(f'class_id:{class_id}, image_id:{image_id}, confidence:{cnf}')
                mask = mutils.decode(sg).astype(bool)
                print_masked_img(image_id, mask)

KeyError: 0

In [22]:
print(annos)

[{'filename': '0040581b-f1f2-4fbe-b043-b6bfea5404bb.jpg', 'width': 2048, 'height': 2048, 'ann': {'bboxes': None, 'labels': None, 'masks': None}}, {'filename': '004a270d-34a2-4d60-bbe4-365fca868193.jpg', 'width': 2048, 'height': 2048, 'ann': {'bboxes': None, 'labels': None, 'masks': None}}, {'filename': '00537262-883c-4b37-a3a1-a4931b6faea5.jpg', 'width': 2048, 'height': 2048, 'ann': {'bboxes': None, 'labels': None, 'masks': None}}]
